In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))


In [19]:
import os
import sys
import json
from algosdk.v2client import algod
from account import *

In [31]:
address, private_key, passphrase = generate_algorand_keypair()

My address: H3V5SSOS7QIEM6FJVMP4HHL4AKZE6WG2ZAQLHUGGJFSEI3ZOF5D5OSVZHI
My private key: eKDVIPXv5m+doMFBndm7zf8bg9o/TJF3hQukwPVKSL8+69lJ0vwQRnipqx/DnXwCsk9Y2sggs9DGSWREby4vRw==
My passphrase: audit final motor young soccer fossil agent script popular grid target wolf dawn patch yard equip vendor beyond alarm apart twist bargain where absent decade


In [32]:
key_data = {
    'address': address,
    'private_key': private_key,
    'passphrase': passphrase
}

with open('../data/key_pair.json', 'w') as f:
    json.dump(key_data, f)

In [38]:

def first_transaction_example(private_key, my_address):
    algod_address = "http://localhost:4001"
    algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
    algod_client = algod.AlgodClient(algod_token, algod_address)
    accoun_info = algod_client.account_info(my_address)
    print(f'account info: {accoun_info}')

In [41]:
first_transaction_example(private_key, address)

account info: {'address': 'H3V5SSOS7QIEM6FJVMP4HHL4AKZE6WG2ZAQLHUGGJFSEI3ZOF5D5OSVZHI', 'amount': 5000000, 'amount-without-pending-rewards': 5000000, 'apps-local-state': [], 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0}, 'assets': [], 'created-apps': [], 'created-assets': [], 'min-balance': 100000, 'pending-rewards': 0, 'reward-base': 27521, 'rewards': 0, 'round': 22213850, 'status': 'Offline', 'total-apps-opted-in': 0, 'total-assets-opted-in': 0, 'total-created-apps': 0, 'total-created-assets': 0}
